In [1]:
from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


In [2]:
ROOT_PATH = "/content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto"

# Model

In [3]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers.core import Dense, Dropout

In [4]:
SEQUENCE_LENGTH = 29
NUM_FEATURES = 1024

In [5]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )

        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [6]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim,
            dropout=0.3
        )

        self.dense_proj = keras.Sequential([
            Dense(dense_dim, activation=tf.nn.gelu),
            Dense(embed_dim)
        ])

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [7]:
def build_transformer_model(
    dense_dim,
    num_heads,
    classes,
    dropout=0
):
    pos_emb = PositionalEmbedding(
        sequence_length=SEQUENCE_LENGTH,
        output_dim=NUM_FEATURES,
    )

    transf_enc = TransformerEncoder(
        embed_dim=NUM_FEATURES,
        dense_dim=dense_dim,
        num_heads=num_heads
    )

    head = Dense(classes, activation='softmax')

    inputs = keras.Input(shape=(None, None))
    embedded = pos_emb(inputs)

    x = transf_enc(embedded)
    x = layers.GlobalMaxPooling1D()(x)
    x = Dropout(dropout)(x)

    outputs = head(x)

    model = keras.Model(inputs, outputs)
    return model

In [8]:
def train_model(
    model,
    train_data,
    train_labels,
    checkpoint_path,
    batch_size=32
):
    checkpoint = keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        save_weights_only=True,
        save_best_only=True,
        verbose=1
    )

    history = model.fit(
        train_data,
        train_labels,
        validation_split=VALIDATION_SPLIT, 
        epochs=EPOCHS,
        batch_size=batch_size, 
        callbacks=[checkpoint], 
        shuffle=True
    )

    return history

In [9]:
def predict_labels(model, data):
    pred = model(data)
    return tf.math.argmax(pred, axis=1).numpy()

In [10]:
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i] == y_hat[i] == 1:
           TP += 1
        if y_hat[i] == 1 and y_actual[i] != y_hat[i]:
           FP += 1
        if y_actual[i] == y_hat[i] == 0:
           TN += 1
        if y_hat[i] == 0 and y_actual[i] != y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

# Data loading

In [11]:
FEATURES_PATH = f"{ROOT_PATH}/features-and-labels/features_all.npy"
LABELS_PATH = f"{ROOT_PATH}/features-and-labels/labels_all.npy"

In [14]:
import numpy as np


with open(FEATURES_PATH, 'rb') as file:
    features = np.load(file)

with open(LABELS_PATH, 'rb') as file:
    labels = np.load(file)

In [13]:
features.shape

(32, 40, 29, 1024)

In [15]:
labels.shape

(32, 40)

In [17]:
TEST_SIZE = 10
NUM_VIDEOS = 40

np.random.seed(0)

idxs = list(range(NUM_VIDEOS))

TEST_VIDEO_IDXS = np.random.choice(idxs, TEST_SIZE, replace=False)
TRAIN_VIDEO_IDXS = np.array([c for c in range(NUM_VIDEOS) if c not in TEST_VIDEO_IDXS])

In [18]:
TEST_VIDEO_IDXS

array([22, 20, 25,  4, 10, 15, 28, 11, 18, 29])

In [32]:
NUM_USERS = 32

user_train_features_dict, user_train_labels_dict = {}, {}
user_test_features_dict, user_test_labels_dict = {}, {}

for user in range(NUM_USERS):
    user_features = features[user, :, :, :]
    user_labels = labels[user, :]

    user_train_features = user_features[TRAIN_VIDEO_IDXS, :, :]
    user_test_features = user_features[TEST_VIDEO_IDXS, :, :]

    user_train_labels = np.expand_dims(user_labels[TRAIN_VIDEO_IDXS], axis=1)
    user_test_labels = np.expand_dims(user_labels[TEST_VIDEO_IDXS], axis=1)

    user_train_features_dict[user] = user_train_features
    user_test_features_dict[user] = user_test_features

    user_train_labels_dict[user] = user_train_labels
    user_test_labels_dict[user] = user_test_labels

In [37]:
print(user_train_features.shape)
print(user_test_features.shape)
print()
print(user_train_labels.shape)
print(user_test_labels.shape)

(30, 29, 1024)
(10, 29, 1024)

(30, 1)
(10, 1)


---

# Training

In [43]:
base_model_checkpoint_path = f"{ROOT_PATH}/models/transformer/checkpoints/base-model-weights.ckpt"
individual_model_checkpoint_path = lambda user: f"{ROOT_PATH}/models/transformer/checkpoints/user_{user}_model.ckpt"

In [ ]:
dense_dim = 4096
num_heads = 1
classes = 2
dropout = 0.4

In [46]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5)
loss = "sparse_categorical_crossentropy",
metrics = ["accuracy"]

EPOCHS = 5
VALIDATION_SPLIT = 4/30
BATCH_SIZE = 2

models_dict = {}
checkpoints_dict = {}

for user in range(NUM_USERS):
    print(f"\nTraining model for user {user}\n")
    checkpoint_path = individual_model_checkpoint_path(user)
    checkpoints_dict[user] = checkpoint_path

    train_features = user_train_features_dict[user]
    train_labels = user_train_labels_dict[user]

    model = build_transformer_model(
        dense_dim, 
        num_heads, 
        classes
    )

    model.load_weights(base_model_checkpoint_path);
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics, 
    )

    train_model(
        model,
        train_features,
        train_labels,
        checkpoint_path, 
        batch_size=BATCH_SIZE
    )

    models_dict[user] = model

Epoch 1/10
13/13 [==============================] - ETA: 0s - loss: 0.7105 - accuracy: 0.6154
Epoch 00001: val_loss improved from inf to 1.08378, saving model to /content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto/models/transformer/checkpoints/user_0_model.ckpt
13/13 [==============================] - 3s 102ms/step - loss: 0.7105 - accuracy: 0.6154 - val_loss: 1.0838 - val_accuracy: 0.0000e+00
Epoch 2/10
10/13 [======================>.......] - ETA: 0s - loss: 0.6765 - accuracy: 0.6500
Epoch 00002: val_loss improved from 1.08378 to 0.50251, saving model to /content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto/models/transformer/checkpoints/user_0_model.ckpt
13/13 [==============================] - 1s 77ms/step - loss: 0.6825 - accuracy: 0.6154 - val_loss: 0.5025 - val_accuracy: 1.0000
Epoch 3/10
11/13 [========================>.....] - ETA: 0s - loss: 0.4991 - accuracy: 0.7273
Epoch 00003: val_loss did not improve from 0.50251
13/13 [==============================] - 0s 

In [49]:
for user, model in models_dict.items():
    print(f"Model for user {user}")

    model.load_weights(checkpoints_dict[user])
    loss, accuracy = model.evaluate(user_test_features_dict[user], user_test_labels_dict[user])

    print(f"Test accuracy: {accuracy:.2f}")
    print()

Model for user 0
1/1 [==============================] - 1s 622ms/step - loss: 0.5932 - accuracy: 0.7000
Test accuracy: 0.70

Model for user 1
1/1 [==============================] - 1s 582ms/step - loss: 0.8706 - accuracy: 0.3000
Test accuracy: 0.30

Model for user 2
1/1 [==============================] - 1s 581ms/step - loss: 1.0045 - accuracy: 0.2000
Test accuracy: 0.20

Model for user 3
1/1 [==============================] - 0s 329ms/step - loss: 0.6395 - accuracy: 0.7000
Test accuracy: 0.70

Model for user 4
1/1 [==============================] - 0s 358ms/step - loss: 0.7241 - accuracy: 0.5000
Test accuracy: 0.50

Model for user 5
1/1 [==============================] - 0s 335ms/step - loss: 0.7419 - accuracy: 0.5000
Test accuracy: 0.50

Model for user 6
1/1 [==============================] - 0s 329ms/step - loss: 0.4276 - accuracy: 0.9000
Test accuracy: 0.90

Model for user 7
1/1 [==============================] - 0s 331ms/step - loss: 0.7982 - accuracy: 0.3000
Test accuracy: 0.30



---